# Muon ML Model using Google Tensorflow

The first step is to load all the required libraries:

In [ ]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Then we load the csv in a pandas dataframe:

In [ ]:
muon_dataframe = pd.read_csv('../MuonPOGAnalysisTemplate/output/bxcut_org.csv')

In [ ]:
muon_dataframe

Set feature columns:

In [ ]:
def preprocess_features(muon_dataframe):
  """Prepares input features from Muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from muon simulations
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = muon_dataframe[
['Event',
 'n_Primitive',
 '1dtPrimitive.id_r',
 '2dtPrimitive.id_r',
 '3dtPrimitive.id_r',
 '4dtPrimitive.id_r',
 '1dtPrimitive.id_eta',
 '2dtPrimitive.id_eta',
 '3dtPrimitive.id_eta',
 '4dtPrimitive.id_eta',
 '1dtPrimitive.id_phi',
 '2dtPrimitive.id_phi',
 '3dtPrimitive.id_phi',
 '4dtPrimitive.id_phi',
 '1dtPrimitive.phiB',
 '2dtPrimitive.phiB',
 '3dtPrimitive.phiB',
 '4dtPrimitive.phiB',
 '1dtPrimitive.quality',
 '2dtPrimitive.quality',
 '3dtPrimitive.quality',
 '4dtPrimitive.quality',
 'delta_phi12',
 'delta_phi13',
 'delta_phi14',
 'delta_phi23',
 'delta_phi24',
 'delta_phi34']]
  processed_features = selected_features.copy()
  return processed_features.astype(np.float32)


Set target columns:

In [ ]:
def preprocess_targets(muon_dataframe):
  """
  Prepares target features (i.e., labels) from muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from the Muon data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets["genParticle.pt"] = muon_dataframe["genParticle.pt"]
  return output_targets.astype(np.float32)

For the training set, we'll use the 60% of the dataset

In [ ]:
training_examples = preprocess_features(muon_dataframe.head(1184))
training_examples.describe()

In [ ]:
training_targets = preprocess_targets(muon_dataframe.head(1184))
training_targets.describe()

And for validation, the remaining data:

In [ ]:
validation_examples = preprocess_features(muon_dataframe.tail(790))
validation_examples.describe()

In [ ]:
validation_targets = preprocess_targets(muon_dataframe.tail(790))
validation_targets.describe()

### Train and Evaluate the Model

Create the function for loading the data in a TensorFlow model.

In [ ]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of multiple features.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    for name in features:
        tensor = features[name]
        tensor = tf.expand_dims(tensor,axis=1)
        features[name] = tensor
    return features, labels

And now we need to create a function to configure feature columns. Right now all columns are numerical, so no worries.

In [ ]:
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

Next, the model function

In [ ]:
def train_model(
    my_optimizer,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a linear regression model of multiple features.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    training_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for validation.
      
  Returns:
    A `LinearRegressor` object trained on the training data.
  """

  periods = 20
  steps_per_period = steps / periods
  
  # Create a linear regressor object.
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(
      training_examples, 
      training_targets["genParticle.pt"], 
      batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(
      training_examples, 
      training_targets["genParticle.pt"], 
      num_epochs=1, 
      shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(
      validation_examples, validation_targets["genParticle.pt"], 
      num_epochs=1, 
      shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically access
  # loss metrics.
  print "Training model..."
  print "RMSE (on training data):"
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period,
    )
    # Take a break and compute predictions.
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print "  period %02d : %0.2f" % (period, training_root_mean_squared_error)
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print "Model training finished."
  print "RMSE (on validation data): %0.2f " % (validation_rmse[-1])

  # Output a graph for prediction-target comparison
  fig, ax = plt.subplots()
  ax.scatter(validation_targets,validation_predictions,edgecolors=(0,0,0))
  ax.plot([validation_predictions.min(), validation_predictions.max()], [validation_predictions.min(), validation_predictions.max()], 'k--', lw=4)
  ax.set_xlabel('Measured P_t(GeV)')
  ax.set_ylabel('Predicted P_t(GeV)')
  ax.set_title("DNN Muon Trigger")
  plt.show()
  del ax

  # Output a graph of loss metrics over periods.
  #fig2, ax2 = plt.subplots()
  #ax2.ylabel("RMSE")
  #ax2.xlabel("Periods")
  #ax2.title("Root Mean Squared Error vs. Periods")
  #ax2.tight_layout()
  #ax2.plot(training_rmse, label="training")
  #ax2.plot(validation_rmse, label="validation")
  #ax2.legend()
  #del ax2


  return linear_regressor

In [ ]:
_ = train_model(
    my_optimizer=tf.train.AdamOptimizer(learning_rate=0.0009),
    steps=4000,
    batch_size=250,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Now try with DNN (Deep Neural Network):

In [ ]:
def train_nn_regression_model(
    my_optimizer,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a neural network regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for validation.
      
  Returns:
    A `DNNRegressor` object trained on the training data.
  """

  periods = 20
  steps_per_period = steps / periods
  
  # Create a DNNRegressor object.
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  dnn_regressor = tf.estimator.DNNRegressor(
      feature_columns=construct_feature_columns(training_examples),
      hidden_units=hidden_units,
      optimizer=my_optimizer
  )
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["genParticle.pt"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["genParticle.pt"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["genParticle.pt"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print "Training model..."
  print "RMSE (on training data):"
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    dnn_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
    for item in training_predictions:
        print item
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print "  period %02d : %0.2f" % (period, training_root_mean_squared_error)
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
    if(period==(periods-1)):
      # Output a graph for prediction-target comparison
      fig, ax = plt.subplots()
      ax.scatter(validation_targets,validation_predictions,edgecolors=(0,0,0))
      ax.plot([validation_targets.min(), validation_targets.max()], [validation_targets.min(), validation_targets.max()], 'k--', lw=4)
      ax.set_xlabel('Measured P_t(GeV)')
      ax.set_ylabel('Predicted P_t(GeV)')
      ax.set_title("DNN Muon Trigger")
      plt.show()

  print "Model training finished."

  print "Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error
  print "Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error

  return dnn_regressor, training_rmse, validation_rmse

In [ ]:
dnn_regressor = train_nn_regression_model(
    my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.008),
    steps=1000,
    batch_size=10,
    hidden_units=[10, 2],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Using other optimizers: the Adagrad modifies the learning rate adaptively for each coefficient, monotonically lowering the effective learning rate.

In [ ]:
_, adagrad_training_losses, adagrad_validation_losses = train_nn_regression_model(
    my_optimizer=tf.train.AdagradOptimizer(learning_rate=0.5),
    steps=1000,
    batch_size=100,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

In [ ]:
_, adam_training_losses, adam_validation_losses = train_nn_regression_model(
    my_optimizer=tf.train.AdamOptimizer(learning_rate=0.009),
    steps=500,
    batch_size=100,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

### Try with stratified shuffle split from SKlearn

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
split = ShuffleSplit(n_splits=8, test_size=0.3, random_state=42)
for train_index, test_index in split.split(muon_dataframe):
    strat_train_set = muon_dataframe.loc[train_index]
    strat_test_set = muon_dataframe.loc[test_index]

In [ ]:
training_examples = preprocess_features(strat_train_set)
training_targets = preprocess_targets(strat_train_set)
validation_examples = preprocess_features(strat_test_set)
validation_targets = preprocess_targets(strat_test_set)

In [ ]:
_, adagrad_training_losses, adagrad_validation_losses = train_nn_regression_model(
    my_optimizer=tf.train.AdagradOptimizer(learning_rate=0.5),
    steps=1000,
    batch_size=100,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Try with Adam Optimizer (different variables):

In [ ]:
_, adam_training_losses, adam_validation_losses = train_nn_regression_model(
    my_optimizer=tf.train.AdamOptimizer(learning_rate=0.031),
    steps=4000,
    batch_size=250,
    hidden_units=[12, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)